In [28]:
import selenium
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import random
from fake_useragent import UserAgent

# Read company list from .csv file

In [59]:
df = pd.read_excel('C:/Users/Acer/Downloads/med_device_comp_list.xlsx')
df

,NAME OF ESTABLISHMENT
0,HORIEN OPTIC (M) SDN BHD
1,AMBU SDN. BHD.
2,HYPERBARIC HEALTH ASIA SDN BHD
3,BEUROTECH SDN BHD
4,TANCAH SDN BHD
...,...
2759,MEDICWORLD SDN BHD
2760,CENTRAL MEDICARE SDN. BHD.
2761,PECCA LEATHER SDN BHD
2762,GALLA DENTAL CORPORATION SDN BHD


In [68]:
comp_list = list(df['NAME OF ESTABLISHMENT'])
comp_list[299:450]

['WORLD GLOVES INTERNATIONAL GROUP SDN. BHD.',
 'DCX TECHNOLOGIES SDN BHD',
 'SHIMADZU MALAYSIA SDN BHD',
 'OMEDIS HEALTHCARE SDN. BHD.',
 'FORWARDMEDCARE SDN BHD',
 'BIOMAX RUBBER INDUSTRIES SDN BHD',
 'SETIA MEDIK SDN BHD',
 'BANDAR KENYALANG SDN. BHD.',
 'AB NYLON TECH SDN BHD',
 'MALAYSIA MICROELECTRONIC SOLUTIONS SDN. BHD.',
 'SPECIALISED EQUIPMENT SDN BHD',
 'AXIAN HEALTHCARE SDN BHD',
 'STERLING ASCENT SDN. BHD.',
 'KEISHA MARINE SERVICES SDN BHD',
 'KHAZANAH JAYA SDN. BHD.',
 'TAN SIN LIAN INDUSTRIES SDN BHD',
 'PUNCAK MEDIK SDN BHD',
 'SCIENCE VALLEY SDN. BHD',
 'PROMED MARKETING SDN BHD',
 'FIRST MEDIXX SDN BHD',
 'EDGE CARES SDN. BHD.',
 'INNOVITA SDN. BHD',
 'TEONG HUAT MEDICAL SDN. BHD.',
 'RIDGE BIOMED SDN BHD',
 'QIAGEN BIOTECHNOLOGY MALAYSIA SDN BHD',
 'SSN MEDICAL PRODUCTS SDN BHD',
 'DENEAST SDN BHD',
 'BIELLDERBERG SDN BHD',
 'SYNTEQ MEDICAL SDN BHD',
 'IDEAL QUALITY SDN BHD',
 '3D GENS SDN. BHD.',
 'NAGASE (MALAYSIA) SDN BHD',
 'SABAMED SUPPLIES SDN BHD',
 'INTELLIP

# Selenium

In [46]:
def apply_random_wait(count):
    """
    Apply some random wait time 
    """
    if count % 300 == 0 and count != 0:
        time.sleep(random.randint(60, 120))
    elif count % 10 == 0 and count != 0:
        time.sleep(random.randint(15, 20))
    else:
        time.sleep(random.randint(7, 15))

In [47]:
chrome_path = r"C:\Users\Acer\OneDrive\Documents\Work\INVOKE\shopee-web-scrape\chromedriver.exe"
opt = webdriver.ChromeOptions()
ua = UserAgent()
user_agent = ua.random
opt.add_argument(f'--user-agent={user_agent}')
service = Service(executable_path=chrome_path)
driver = webdriver.Chrome(service=service,options=opt)
base_url = 'https://www.mydata-ssm.com.my/login'
driver.get(base_url)
wait = WebDriverWait(driver, 30)
driver.maximize_window()
username = driver.find_element(By.XPATH,'//*[@id="mat-input-0"]')
username.send_keys("safwansar")
password = driver.find_element(By.XPATH,'//*[@id="mat-input-1"]')
password.send_keys("SAFWANshamsir")
login = driver.find_element(By.XPATH,'//*[@id="recaptcha2"]/button')
login.click()
try:
   time.sleep(5)
   webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
except:
   pass
time.sleep(5)

## Access company name

In [48]:
data=str()
comp_search = []
comp_num = []
comp_name = []
comp_type = []

for i, comp in enumerate(comp_list[:150]):
    search = driver.find_element(By.CLASS_NAME,'form-control')
    search.send_keys(comp)
    webdriver.ActionChains(driver).send_keys(Keys.ENTER).perform()
    apply_random_wait(i)

    data += driver.page_source
    
    # Company to validate
    search_comp = comp
    comp_search.append(search_comp)

    # Company name
    try:
        name_comp = driver.find_element(By.CSS_SELECTOR, 'mat-cell.mat-cell.cdk-column-name.mat-column-name.ng-star-inserted').text.replace('Name\n','')
    except:
        name_comp = "No match"
    comp_name.append(name_comp)

    # Company type
    try:
        type_comp = driver.find_element(By.CSS_SELECTOR, 'mat-cell.mat-cell.cdk-column-entityType.mat-column-entityType.ng-star-inserted').text.replace('Type\n','') 
    except:
        type_comp = "No match"
    comp_type.append(type_comp)
    apply_random_wait(i)

    # Clear the search bar
    search.clear()




# Convert into dataframe 

In [49]:
len(comp_name)

206

In [50]:
scrape_data = pd.DataFrame()

scrape_data['Company Verification'] = comp_search
scrape_data['Company Name']= comp_name
scrape_data['Company Type'] = comp_type
scrape_data

,Company Verification,Company Name,Company Type
0,F.A. Herbs Sdn. Bhd,No match,No match
1,F.E. Venture Marketing Sdn. Bhd.,No match,No match
2,Fabullite Food & Supplement Sdn. Bhd.,FABULLITE FOOD & SUPPLEMENT SDN. BHD.,Company
3,Fair-Fairly Enterprise,No match,No match
4,Fasiliti Penyediaan Radiofarmaseutikal - Siklo...,No match,No match
...,...,...,...
201,Yi Wang Honey Garden (M) Sdn. Bhd.,YI WANG HONEY GARDEN (M) SDN. BHD.,Company
202,Yokness Sdn. Bhd.,YOKNESS SDN. BHD.,Company
203,YPL Healthcare Product Sdn. Bhd.,YPL HEALTHCARE PRODUCT SDN. BHD.,Company
204,Zhulian Nutraceutical Sdn. Bhd.,ZHULIAN NUTRACEUTICAL SDN. BHD.,Company


## Example for one company (Selenium)

In [66]:
reg_num = driver.find_element(By.CSS_SELECTOR, 'mat-row mat-cell:first-child a.ng-star-inserted').text 
reg_num

'198701001694 (160363-M)'

In [69]:
name_comp = driver.find_element(By.CSS_SELECTOR, 'mat-cell.mat-cell.cdk-column-name.mat-column-name.ng-star-inserted').text.replace('Name\n','')
name_comp

'21ST CENTURY PRODUCTS SDN. BHD.'

In [71]:
type_comp = driver.find_element(By.CSS_SELECTOR, 'mat-cell.mat-cell.cdk-column-entityType.mat-column-entityType.ng-star-inserted').text.replace('Type\n','')
type_comp

'Company'

# BeautifulSoup (to process comp_num[Registration number])

In [51]:
soup = BeautifulSoup(data, 'html.parser')
row_data = soup.find_all('mat-row','example-element-row')

records = []
for row in row_data:
    #comp_search = company_name
    comp_num = row.find('a','ng-star-inserted').text.strip()
    #comp_name = row.find('mat-cell','mat-cell cdk-column-name mat-column-name ng-star-inserted').text.strip().replace('Name','')
    #comp_type = row.find('mat-cell','mat-cell cdk-column-entityType mat-column-entityType ng-star-inserted').text.strip().replace('Type','')

    company_info = (comp_num) #, comp_name, comp_type

    records.append(company_info)

In [52]:
records

['201501038623 (1163944-K)',
 '197701004150 (35028-W)',
 '200101031324 (567083-H)',
 '200401031753 (670261-M)',
 '201201001151 (974675-H)',
 '201301026277 (1056107-A)',
 '200601017074 (736826-H)',
 '199203077564 (AS0142679-X)',
 '200101005098 (540854-H)',
 '201301044553 (1074376-V)',
 '199601005815 (378161-T)',
 '200301035033 (637454-M)',
 '198801001527 (168884-H)',
 '198901005322 (182627-W)',
 '201103146087 (002050811-V)',
 '200501021138 (703268-D)',
 '199803005755 (001134717-H)',
 '200401028758 (667266-X)',
 '198501006475 (138919-K)',
 '197901006271 (50554-D)',
 '198701000341 (159007-X)',
 '198801007471 (174828-M)',
 '199601024539 (396891-K)',
 '198001004691 (58476-A)',
 '199501007902 (337100-K)',
 '200501024986 (707119-W)',
 '200401023395 (661901-P)',
 '200401011627 (650130-X)',
 '200301012169 (614589-A)',
 '201101007989 (936128-V)',
 '196703003111 (000229739-V)',
 '200301017140 (619560-W)',
 '198201010358 (90082-V)',
 '200401030191 (668699-P)',
 '199401010563 (296242-K)',
 '2003010

In [53]:
comp_number = []
j = 0
for i in range(len(comp_name)):
    if comp_name[i] == 'No match':
        comp_number.append('No match')
    else:
        comp_number.append(records[j])
        j += 1
comp_number

['No match',
 'No match',
 '201501038623 (1163944-K)',
 'No match',
 'No match',
 '197701004150 (35028-W)',
 '200101031324 (567083-H)',
 '200401031753 (670261-M)',
 '201201001151 (974675-H)',
 '201301026277 (1056107-A)',
 '200601017074 (736826-H)',
 'No match',
 '199203077564 (AS0142679-X)',
 '200101005098 (540854-H)',
 '201301044553 (1074376-V)',
 '199601005815 (378161-T)',
 '200301035033 (637454-M)',
 '198801001527 (168884-H)',
 '198901005322 (182627-W)',
 '201103146087 (002050811-V)',
 '200501021138 (703268-D)',
 '199803005755 (001134717-H)',
 '200401028758 (667266-X)',
 '198501006475 (138919-K)',
 '197901006271 (50554-D)',
 'No match',
 '198701000341 (159007-X)',
 '198801007471 (174828-M)',
 'No match',
 'No match',
 'No match',
 '199601024539 (396891-K)',
 '198001004691 (58476-A)',
 '199501007902 (337100-K)',
 '200501024986 (707119-W)',
 '200401023395 (661901-P)',
 '200401011627 (650130-X)',
 '200301012169 (614589-A)',
 'No match',
 'No match',
 '201101007989 (936128-V)',
 '196703

In [54]:
len(comp_number), len(comp_name)

(206, 206)

In [55]:
scrape_data['Registration Number'] = comp_number
scrape_data

,Company Verification,Company Name,Company Type,Registration Number
0,F.A. Herbs Sdn. Bhd,No match,No match,No match
1,F.E. Venture Marketing Sdn. Bhd.,No match,No match,No match
2,Fabullite Food & Supplement Sdn. Bhd.,FABULLITE FOOD & SUPPLEMENT SDN. BHD.,Company,201501038623 (1163944-K)
3,Fair-Fairly Enterprise,No match,No match,No match
4,Fasiliti Penyediaan Radiofarmaseutikal - Siklo...,No match,No match,No match
...,...,...,...,...
201,Yi Wang Honey Garden (M) Sdn. Bhd.,YI WANG HONEY GARDEN (M) SDN. BHD.,Company,199401013024 (298703-W)
202,Yokness Sdn. Bhd.,YOKNESS SDN. BHD.,Company,201201040713 (1025191-K)
203,YPL Healthcare Product Sdn. Bhd.,YPL HEALTHCARE PRODUCT SDN. BHD.,Company,199101016741 (227053-V)
204,Zhulian Nutraceutical Sdn. Bhd.,ZHULIAN NUTRACEUTICAL SDN. BHD.,Company,201701040362 (1254535-K)


In [56]:
# Read mydata_ssm.xlsx file
#mydata_df = pd.read_excel('C:/Users/Acer/Downloads/reg_med_device_ssm.xlsx')

# Concatenate 'scrape_data' dataframe with 'df'
#mydata_df = pd.concat([mydata_df, scrape_data], ignore_index=True)

# Write the updated dataframe back to 'reg_med_device_ssm.xlsx' mydata_df
scrape_data.to_excel('C:/Users/Acer/Downloads/reg_med_device_ssm.xlsx', index=False)

## Example for one row (BeautifulSoup)

In [32]:
row = row_data[0]
row

<mat-row _ngcontent-c21="" class="example-element-row mat-row ng-star-inserted" role="row"><!-- --><mat-cell _ngcontent-c21="" class="mat-cell cdk-column-entityNumber mat-column-entityNumber ng-star-inserted" role="gridcell"><span _ngcontent-c21="" class="mobile-label">Number<br _ngcontent-c21=""/></span><!-- --><a _ngcontent-c21="" class="ng-star-inserted"> 200501033350 (715490-M)</a><!-- --></mat-cell><mat-cell _ngcontent-c21="" class="mat-cell cdk-column-name mat-column-name ng-star-inserted" role="gridcell"><span _ngcontent-c21="" class="mobile-label">Name<br _ngcontent-c21=""/></span>CCB MEDICAL DEVICES SDN. BHD. </mat-cell><mat-cell _ngcontent-c21="" class="mat-cell cdk-column-entityType mat-column-entityType ng-star-inserted" role="gridcell"><span _ngcontent-c21="" class="mobile-label">Type<br _ngcontent-c21=""/></span>Company </mat-cell></mat-row>

In [33]:
comp_num = row.find('a','ng-star-inserted').text.strip()
comp_num

'200501033350 (715490-M)'

In [36]:
comp_name = row.find('mat-cell','mat-cell cdk-column-name mat-column-name ng-star-inserted').text.strip().replace('Name', '')
#comp_name = comp_name.replace('Name', '')
comp_name

'CCB MEDICAL DEVICES SDN. BHD.'

In [34]:
comp_type = row.find('mat-cell','mat-cell cdk-column-entityType mat-column-entityType ng-star-inserted').text.strip().replace('Type', '')
#comp_type = comp_type.replace('Type', '')
comp_type

'Company'